In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

In [ ]:
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f tdcsfog_metadata.csv
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f defog_metadata.csv
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f daily_metadata.csv
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f events.csv
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f tasks.csv
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f subjects.csv
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f sample_submission.csv

In [4]:
import pandas as pd
import glob
import zipfile
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


### Create DataFrames (Metadata)

Δημιουργία DataFrames για τα αρχεία που αφορούν τα metadata.
* tdcsfog_metadata.csv
* defog_metadata.csv
* daily_metadata.csv
* subjects.csv
* tasks.csv
* events.csv

In [5]:
tdcsfog_metadata = pd.read_csv('/content/tdcsfog_metadata.csv')
defog_metadata = pd.read_csv('/content/defog_metadata.csv')
daily_metadata = pd.read_csv('/content/daily_metadata.csv')
subjects = pd.read_csv('/content/subjects.csv')
tasks = pd.read_csv('/content/tasks.csv')
events = pd.read_csv('/content/events.csv')

In [6]:
tdcsfog_metadata

,Id,Subject,Visit,Test,Medication
0,003f117e14,4dc2f8,3,2,on
1,009ee11563,f62eec,4,2,on
2,011322847a,231c3b,2,2,on
3,01d0fe7266,231c3b,2,1,off
4,024418ba39,fa8764,19,3,on
...,...,...,...,...,...
828,feba449e1a,312788,19,1,on
829,ff4f844fd3,af82b2,2,3,on
830,ff53514514,3b2403,2,3,on
831,ff92d9244d,2d57c2,20,2,on


In [7]:
defog_metadata

,Id,Subject,Visit,Medication
0,02ab235146,e1f62e,2,on
1,02ea782681,ae2d35,2,on
2,06414383cf,8c1f5e,2,off
3,092b4c1819,2874c5,1,off
4,0a900ed8a2,0e3d49,2,on
...,...,...,...,...
132,f3a921edee,1a778d,1,off
133,f40e8c6ebe,575c60,1,off
134,f8ddbdd98d,107712,1,on
135,f9efef91fb,5d9cae,2,off


Concatenation of tdcsfog_metadata and defog_metadata

In [8]:
tdcsfog_metadata['Type'] = 'tdcsfog'
defog_metadata['Type'] = 'defog'
metadata = pd.concat([tdcsfog_metadata, defog_metadata])
metadata['Medication'] = metadata['Medication'].factorize()[0] # on medication = 0 off medication =1
metadata

,Id,Subject,Visit,Test,Medication,Type
0,003f117e14,4dc2f8,3,2.0,0,tdcsfog
1,009ee11563,f62eec,4,2.0,0,tdcsfog
2,011322847a,231c3b,2,2.0,0,tdcsfog
3,01d0fe7266,231c3b,2,1.0,1,tdcsfog
4,024418ba39,fa8764,19,3.0,0,tdcsfog
...,...,...,...,...,...,...
132,f3a921edee,1a778d,1,NaN,1,defog
133,f40e8c6ebe,575c60,1,NaN,1,defog
134,f8ddbdd98d,107712,1,NaN,0,defog
135,f9efef91fb,5d9cae,2,NaN,1,defog


In [9]:
daily_metadata #not important info

,Id,Subject,Visit,Beginning of recording [00:00-23:59]
0,00c4c9313d,fba3a3,1,10:19
1,07a96f89ec,7da72f,1,07:30
2,0d1bc672a8,056372,2,08:30
3,0e333c9833,b4bd22,1,11:30
4,164adaed7b,9f72eb,1,13:00
...,...,...,...,...
60,e658b0aa3d,0e3d49,1,08:00
61,ed0a487f20,9404ad,1,12:30
62,ef1db3ca64,12f8d1,1,08:00
63,f16c5cda55,2874c5,1,08:30


In [10]:
subjects

,Subject,Visit,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
0,00f674,2.0,63,M,27.0,43.0,49.0,24
1,00f674,1.0,63,M,27.0,31.0,30.0,26
2,02bc69,NaN,69,M,4.0,21.0,NaN,22
3,040587,2.0,75,M,26.0,52.0,69.0,21
4,040587,1.0,75,M,26.0,47.0,75.0,24
...,...,...,...,...,...,...,...,...
168,f80507,1.0,57,M,2.0,12.0,NaN,0
169,fa8764,NaN,60,F,7.0,30.0,NaN,19
170,fba3a3,1.0,65,F,8.0,28.0,NaN,0
171,fcb9f5,1.0,69,M,3.5,27.0,49.0,23


Join  metadata and subjects

In [11]:
subjects['Sex'] = subjects['Sex'].factorize()[0] #Males=0,Females=1
subjects = subjects.fillna(0).groupby('Subject').median() #fill NA with 0 and grouped and median (we can change those)
metadata_join_subjects = metadata.merge(subjects, how='left', on='Subject').copy()
metadata_join_subjects

,Id,Subject,Visit_x,Test,Medication,Type,Visit_y,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
0,003f117e14,4dc2f8,3,2.0,0,tdcsfog,0.0,68.0,1.0,9.0,17.0,15.0,15.0
1,009ee11563,f62eec,4,2.0,0,tdcsfog,0.0,71.0,0.0,10.0,42.0,0.0,24.0
2,011322847a,231c3b,2,2.0,0,tdcsfog,0.0,67.0,0.0,12.0,27.0,28.0,19.0
3,01d0fe7266,231c3b,2,1.0,1,tdcsfog,0.0,67.0,0.0,12.0,27.0,28.0,19.0
4,024418ba39,fa8764,19,3.0,0,tdcsfog,0.0,60.0,1.0,7.0,30.0,0.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,f3a921edee,1a778d,1,NaN,1,defog,1.5,65.0,0.0,7.0,50.0,59.5,24.5
966,f40e8c6ebe,575c60,1,NaN,1,defog,1.0,28.0,0.0,4.0,54.0,50.0,25.0
967,f8ddbdd98d,107712,1,NaN,0,defog,1.0,82.0,1.0,11.0,38.0,42.0,21.0
968,f9efef91fb,5d9cae,2,NaN,1,defog,1.5,72.0,0.5,14.0,22.5,39.0,16.0


In [12]:
tasks

,Id,Begin,End,Task
0,02ab235146,10.000,190.480,Rest1
1,02ab235146,211.240,271.560,Rest2
2,02ab235146,505.880,522.400,4MW
3,02ab235146,577.960,594.640,4MW-C
4,02ab235146,701.320,715.280,MB1
...,...,...,...,...
2812,f9fc61ce85,800.586,810.714,TUG-DT
2813,f9fc61ce85,904.416,926.181,Turning-ST
2814,f9fc61ce85,959.211,989.855,Turning-DT
2815,f9fc61ce85,1087.313,1105.086,Hotspot1


In [13]:
tasks['Duration'] = tasks['End'] - tasks['Begin']
tasks = pd.pivot_table(tasks, values=['Duration'], index=['Id'], columns=['Task'], aggfunc='sum', fill_value=0)
tasks.columns = [c[1] for c in tasks.columns]
tasks = tasks.reset_index()
tasks

,Id,4MW,4MW-C,Hotspot1,Hotspot1-C,Hotspot2,Hotspot2-C,MB1,MB10,MB11,...,MB8,MB9,Rest1,Rest2,TUG-C,TUG-DT,TUG-ST,Turning-C,Turning-DT,Turning-ST
0,02ab235146,16.520,16.680,16.760,16.240,53.920,64.600,13.960,17.960,17.400,...,12.840,30.800,180.48,60.32,38.440,47.920,36.240,21.920,46.400,23.320
1,02ea782681,11.618,11.796,11.525,11.692,8.329,9.032,3.469,6.624,6.230,...,11.849,30.650,0.00,0.00,18.343,19.932,20.130,18.042,21.588,18.698
2,06414383cf,24.860,41.584,25.885,0.000,38.642,0.000,0.000,0.000,0.000,...,0.000,0.000,0.00,0.00,0.000,44.832,33.867,0.000,83.837,124.299
3,092b4c1819,13.664,0.000,15.409,0.000,34.834,0.000,0.000,0.000,0.000,...,0.000,0.000,0.00,0.00,0.000,30.166,23.347,0.000,45.025,39.351
4,0a900ed8a2,11.720,11.840,10.600,10.720,43.171,41.160,1.760,21.040,10.560,...,31.040,30.520,180.88,60.32,18.083,25.000,19.920,18.699,18.200,16.880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,f3a921edee,16.722,0.000,16.383,0.000,76.200,0.000,0.000,0.000,0.000,...,0.000,0.000,0.00,0.00,0.000,29.043,29.820,0.000,136.840,76.960
133,f40e8c6ebe,12.867,0.000,27.906,0.000,152.333,0.000,0.000,0.000,0.000,...,0.000,0.000,0.00,0.00,0.000,36.805,28.232,0.000,98.579,69.889
134,f8ddbdd98d,23.787,0.000,32.556,0.000,38.557,0.000,6.347,10.356,10.655,...,0.000,33.289,0.00,0.00,0.000,44.132,54.993,0.000,100.433,72.152
135,f9efef91fb,16.351,17.552,16.155,15.166,67.059,51.715,0.000,0.000,0.000,...,0.000,0.000,0.00,0.00,23.483,23.541,22.137,36.898,38.632,30.831


In [14]:
events

,Id,Init,Completion,Type,Kinetic
0,003f117e14,8.61312,14.7731,Turn,1.0
1,009ee11563,11.38470,41.1847,Turn,1.0
2,009ee11563,54.66470,58.7847,Turn,1.0
3,011322847a,28.09660,30.2966,Turn,1.0
4,01d0fe7266,30.31840,31.8784,Turn,1.0
...,...,...,...,...,...
3539,f9fc61ce85,620.74000,623.8450,Walking,0.0
3540,f9fc61ce85,774.67800,774.8330,Walking,1.0
3541,f9fc61ce85,924.11900,925.6270,Turn,1.0
3542,f9fc61ce85,983.03900,984.0380,Turn,0.0


In [15]:
metadata_join_subjects_join_events = metadata_join_subjects.merge(events, how='left', on='Id').copy()
metadata_join_subjects_join_events

,Id,Subject,Visit_x,Test,Medication,Type_x,Visit_y,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ,Init,Completion,Type_y,Kinetic
0,003f117e14,4dc2f8,3,2.0,0,tdcsfog,0.0,68.0,1.0,9.0,17.0,15.0,15.0,8.61312,14.7731,Turn,1.0
1,009ee11563,f62eec,4,2.0,0,tdcsfog,0.0,71.0,0.0,10.0,42.0,0.0,24.0,11.38470,41.1847,Turn,1.0
2,009ee11563,f62eec,4,2.0,0,tdcsfog,0.0,71.0,0.0,10.0,42.0,0.0,24.0,54.66470,58.7847,Turn,1.0
3,011322847a,231c3b,2,2.0,0,tdcsfog,0.0,67.0,0.0,12.0,27.0,28.0,19.0,28.09660,30.2966,Turn,1.0
4,01d0fe7266,231c3b,2,1.0,1,tdcsfog,0.0,67.0,0.0,12.0,27.0,28.0,19.0,30.31840,31.8784,Turn,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3974,f9fc61ce85,040587,1,NaN,0,defog,1.5,75.0,0.0,26.0,49.5,72.0,22.5,620.74000,623.8450,Walking,0.0
3975,f9fc61ce85,040587,1,NaN,0,defog,1.5,75.0,0.0,26.0,49.5,72.0,22.5,774.67800,774.8330,Walking,1.0
3976,f9fc61ce85,040587,1,NaN,0,defog,1.5,75.0,0.0,26.0,49.5,72.0,22.5,924.11900,925.6270,Turn,1.0
3977,f9fc61ce85,040587,1,NaN,0,defog,1.5,75.0,0.0,26.0,49.5,72.0,22.5,983.03900,984.0380,Turn,0.0


### Create DataFrames (recordings data)

Λήψη αρχείων καταγραφών tdcsfog που θα συνθέσουν το trainset

Download tdcsfog data used in preprocessing

In [ ]:
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/0330ea6680.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/0506d9a39f.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/03e7159bcc.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/04c2e97800.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/0526a6a0fd.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/02edc527c0.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/009ee11563.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/011322847a.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/04b449020a.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/0494003044.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/01d0fe7266.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/04e10e0797.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/02e8454f57.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/024ba3ebd5.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/03bb0c0c46.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/0383be6b43.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/052a913d06.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/03c9d8dbfa.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/003f117e14.csv' -p /content/train/tdcsfog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f 'train/tdcsfog/024418ba39.csv' -p /content/train/tdcsfog

Read csv files and create the dataframe for the tdcsfog

In [17]:
# folder where tdcsfog csv data exist - multiple csv files
folder_path = '/content/train/tdcsfog/'

# Get a list of all CSV files in the folder
file_list = glob.glob(folder_path + '*.csv')

# Initialize an empty list to store the DataFrames
dataframes = []

# Iterate over the file list and read each CSV file into a DataFrame
for file in file_list:
    df = pd.read_csv(file)
    df["Id"]=file.split("/")[4].split(".")[0]
    dataframes.append(df)

# Concatenate the DataFrames into a single DataFrame
tdcsfog_df = pd.concat(dataframes, ignore_index=True)

In [18]:
tdcsfog_df

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Id
0,0,-9.358670,0.601826,-2.876796,0,0,0,0494003044
1,1,-9.362253,0.599822,-2.873079,0,0,0,0494003044
2,2,-9.365474,0.601983,-2.878293,0,0,0,0494003044
3,3,-9.359846,0.598368,-2.876051,0,0,0,0494003044
4,4,-9.360061,0.598931,-2.874567,0,0,0,0494003044
...,...,...,...,...,...,...,...,...
135675,13618,-9.352971,0.815884,-1.868772,0,0,0,03c9d8dbfa
135676,13619,-9.377743,0.814042,-1.871012,0,0,0,03c9d8dbfa
135677,13620,-9.386903,0.804841,-1.905264,0,0,0,03c9d8dbfa
135678,13621,-9.379790,0.793695,-1.920870,0,0,0,03c9d8dbfa


Λήψη αρχείων καταγραφών defog που θα συνθέσουν το trainset

Download defog data files

In [ ]:
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/0c55be4384.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/2c12284ed2.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/1ff78d55e9.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/139f60d29b.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/1d99c2eecf.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/13a4fe5159.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/0eaac04f17.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/2e75cf4507.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/0ec76d2d8e.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/15508c7f41.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/06414383cf.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/0d7ab3a9f9.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/2b6a1c294a.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/28209b9006.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/32843e32b6.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/02ea782681.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/092b4c1819.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/18e7abc37e.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/2a01c919c0.csv' -p /content/train/defog
! kaggle competitions download tlvmc-parkinsons-freezing-gait-prediction -f  'train/defog/32d03020a9.csv' -p /content/train/defog

Unzip defog data files

In [20]:
# folder where defog zipped csv data exist - multiple zip files
folder_path = '/content/train/defog/'

# Get a list of all ZIP files in the folder
zip_file_list = glob.glob(folder_path + '*.zip')

# Unzip all the files in the folder
for zip_file in zip_file_list:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(folder_path)
    # Remove the zip file after extracting
    os.remove(zip_file)

Read defog csv files and create a Dataframe

In [21]:
# folder where defog (unzipped) csv data exist - multiple csv files
folder_path = '/content/train/defog/'

# Get a list of all CSV files in the folder
file_list = glob.glob(folder_path + '*.csv')

# Initialize an empty list to store the DataFrames
dataframes = []

# Iterate over the file list and read each CSV file into a DataFrame
for file in file_list:
    df = pd.read_csv(file)
    df["Id"]=file.split("/")[4].split(".")[0]
    dataframes.append(df)

# Concatenate the DataFrames into a single DataFrame
defog_df = pd.concat(dataframes, ignore_index=True)

In [22]:
defog_df

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task,Id
0,0,-0.906250,0.046875,-0.465332,0,0,0,False,False,32843e32b6
1,1,-0.906250,0.046875,-0.456310,0,0,0,False,False,32843e32b6
2,2,-0.906250,0.046875,-0.465798,0,0,0,False,False,32843e32b6
3,3,-0.906250,0.046875,-0.455838,0,0,0,False,False,32843e32b6
4,4,-0.906250,0.046875,-0.468750,0,0,0,False,False,32843e32b6
...,...,...,...,...,...,...,...,...,...,...
3068848,182655,-0.984041,0.068054,-0.173262,0,0,0,False,False,092b4c1819
3068849,182656,-0.986285,0.067161,-0.171767,0,0,0,False,False,092b4c1819
3068850,182657,-0.985703,0.068139,-0.171534,0,0,0,False,False,092b4c1819
3068851,182658,-0.987393,0.070450,-0.173982,0,0,0,False,False,092b4c1819


Ένωση DataFrames (tdcsfog + defog). Το column Test που δεν υπάρχει στο defog θα λάβει NaN values

Concatenate tdcsfog & defog dataframes

In [23]:
all_data_df = pd.concat( [tdcsfog_df, defog_df], ignore_index=True)

In [24]:
all_data_df

/usr/local/lib/python3.10/dist-packages/IPython/lib/pretty.py:700: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/usr/local/lib/python3.10/dist-packages/google/colab/_interactive_table_hint_button.py:162: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_html=dataframe._repr_html_(),


,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Id,Valid,Task
0,0,-9.358670,0.601826,-2.876796,0,0,0,0494003044,NaN,NaN
1,1,-9.362253,0.599822,-2.873079,0,0,0,0494003044,NaN,NaN
2,2,-9.365474,0.601983,-2.878293,0,0,0,0494003044,NaN,NaN
3,3,-9.359846,0.598368,-2.876051,0,0,0,0494003044,NaN,NaN
4,4,-9.360061,0.598931,-2.874567,0,0,0,0494003044,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3204528,182655,-0.984041,0.068054,-0.173262,0,0,0,092b4c1819,False,False
3204529,182656,-0.986285,0.067161,-0.171767,0,0,0,092b4c1819,False,False
3204530,182657,-0.985703,0.068139,-0.171534,0,0,0,092b4c1819,False,False
3204531,182658,-0.987393,0.070450,-0.173982,0,0,0,092b4c1819,False,False


join all_data_df with previous metadata

In [25]:
all_data_df=pd.merge(all_data_df,metadata, how='left', on='Id')
del all_data_df['Task'] #not useful
del all_data_df['Visit'] #not useful
all_data_df.fillna(0) #only NAN in test column which corespond to 0

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Id,Valid,Subject,Test,Medication,Type
0,0,-9.358670,0.601826,-2.876796,0,0,0,0494003044,0,e9fc55,3.0,0,tdcsfog
1,1,-9.362253,0.599822,-2.873079,0,0,0,0494003044,0,e9fc55,3.0,0,tdcsfog
2,2,-9.365474,0.601983,-2.878293,0,0,0,0494003044,0,e9fc55,3.0,0,tdcsfog
3,3,-9.359846,0.598368,-2.876051,0,0,0,0494003044,0,e9fc55,3.0,0,tdcsfog
4,4,-9.360061,0.598931,-2.874567,0,0,0,0494003044,0,e9fc55,3.0,0,tdcsfog
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3204528,182655,-0.984041,0.068054,-0.173262,0,0,0,092b4c1819,False,2874c5,0.0,1,defog
3204529,182656,-0.986285,0.067161,-0.171767,0,0,0,092b4c1819,False,2874c5,0.0,1,defog
3204530,182657,-0.985703,0.068139,-0.171534,0,0,0,092b4c1819,False,2874c5,0.0,1,defog
3204531,182658,-0.987393,0.070450,-0.173982,0,0,0,092b4c1819,False,2874c5,0.0,1,defog
